In [96]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from classy import Class
from scipy.optimize import fsolve
from scipy.interpolate import interp1d
from scipy.integrate import odeint
import math


In [97]:
font = {'size'   : 10, 'family':'STIXGeneral'}
axislabelfontsize='small'
matplotlib.rc('font', **font)
matplotlib.mathtext.rcParams['legend.fontsize']='medium'

In [26]:
LCDM_settings = {# wich output? ClTT, transfer functions delta_i and theta_i
                   'output':'tCl,pCl,lCl,mPk',
                   'lensing':'yes',
                   # LambdaCDM parameters
                   'h':0.6821,
                   'omega_b':0.02253,
                   'omega_cdm':0.1177,
                   'A_s':2.216e-9, #caught a typo
                   'n_s':0.9686,
                   'tau_reio':0.085,
                   'm_ncdm': 0.06,
                   'N_ncdm':1, 
                   'N_ur':2.0328,
                   #'Omega_Lambda': 6.878622e-01,
                   # Take fixed value for primordial Helium (instead of automatic BBN adjustment)
                   #'YHe':0.246,
                   # other output and precision parameters
                   'l_max_scalars':2508,
                   'non linear':'HMcode',
                   'P_k_max_h/Mpc':100,
                   'z_max_pk':3.}

In [27]:
LCDM = Class()
LCDM.set(LCDM_settings)
LCDM.compute()

In [28]:
pkLCDM, kLCDM, zLCDM = LCDM.get_pk_and_k_and_z()

In [29]:
zDES = np.array([.225,.375,.525,.675,.825])

In [30]:
# From smith page 17:


In [98]:
EDE = Class()
#  Use best-fit params  of Smith+ :
EDE.set({'h': .7219})
EDE.set({'f_scf': 3.98e+26, 'm_scf': 5.301e-28,'thetai_scf': 2.83, 'A_s': 2.215e-09, 'n_s': 0.9889, 'omega_b': 0.02253, 'omega_cdm': 0.1306, 'm_ncdm': 0.06, 'tau_reio': 0.072})

#'non linear':'halofit', HMCODE
EDE.set({'non linear':'HMCODE','N_ncdm':1, 'N_ur':2.0328, 'Omega_Lambda':0.0, 'Omega_fld':0, 'Omega_scf':-1, 'n_scf':3, 'CC_scf':1, 'scf_parameters':'1, 1, 1, 1, 1, 0.0', 'scf_tuning_index':3, 'attractor_ic_scf':'no', 'output':'tCl pCl lCl mPk', 'lensing':'yes', 'l_max_scalars':2508,'P_k_max_h/Mpc':200,'z_max_pk':3.})
EDE.compute()

In [99]:
print EDE.theta_s_100()
print EDE.fEDE()
print np.log10(EDE.z_c())

1.04142144065
0.12235075021
3.562732440197178


In [20]:
# sigma8(z)
print LCDM.sigma8()
print EDE.sigma8()

ratiosigma8 = np.zeros(100)
zsigma8=np.zeros(100)
for i in range(100):
    ratiosigma8[i] = EDE.sigma(8/EDE.h(),i/100.)/LCDM.sigma(8/LCDM.h(),i/100.)
    zsigma8[i]=i/100.

plt.plot(zsigma8,ratiosigma8,color="k",lw=2,label='$\sigma_8 ^{EDE}/\sigma_8 ^{LCDM} (z) $')
plt.hlines(0, 0, 1, colors='k', linestyles='dashed')
plt.grid()
#plt.xscale('log')
plt.xlabel(r'$z$')
plt.ylabel(r'$ \sigma_8 ^{EDE}/\sigma_8 ^{LCDM} (z) $')
plt.xlim([0,1])
plt.ylim([1,1.06])
#plt.vlines(1./(8./EDE.h()), -.25, .05, colors='r', linestyles='dashed')
plt.legend(loc='upper center')
plt.savefig('ratiosigma8.pdf')
plt.clf()

# fsigma8(z)

ratiofsigma8 = np.zeros(100)

for i in range(100):
    ratiofsigma8[i] = EDE.get_fsigma8(i/100.)/LCDM.get_fsigma8(i/100.)


plt.plot(zsigma8,ratiofsigma8,color="k",lw=2,label='$f\sigma_8 ^{EDE}/f\sigma_8 ^{LCDM} (z) $')
plt.hlines(0, 0, 1, colors='k', linestyles='dashed')
plt.grid()
#plt.xscale('log')
plt.xlabel(r'$z$')
plt.ylabel(r'$ f\sigma_8 ^{EDE}/f\sigma_8 ^{LCDM} (z) $')
plt.xlim([0,1])
plt.ylim([1,1.06])
#plt.vlines(1./(8./EDE.h()), -.25, .05, colors='r', linestyles='dashed')
plt.legend(loc='upper center')
plt.savefig('ratiofsigma8.pdf')
plt.clf()


0.825167183833
0.849774576328


In [32]:
kLCDM[-1]/LCDM.h()

105.0597848654366

In [34]:
#P(k) in units of (h/Mpc)^-3 and k in units of (h/Mpc)
pkzLCDM = np.zeros((len(kLCDM)-2,len(zDES)))
pkzEDE = np.zeros((len(kLCDM)-2,len(zDES)))
pkzDiff= np.zeros((len(kLCDM)-2,len(zDES)))
pkzratio= np.zeros((len(kLCDM)-2,len(zDES)))
kLCDMhLCDM=np.zeros(len(kLCDM)-2)
kLCDMhEDE=np.zeros(len(kLCDM)-2)
for kindex in range(len(kLCDM)-2):
    for zindex in range(len(zDES)):
        kLCDMhLCDM[kindex]=kLCDM[kindex]/LCDM.h()
        kLCDMhEDE[kindex]=kLCDM[kindex]/EDE.h()
        #pkzLCDM[kindex,zindex] = LCDM.h()**3*LCDM.pk(kLCDM[kindex]/LCDM.h(),zDES[zindex])
        #pkzEDE[kindex,zindex] = EDE.h()**3*EDE.pk(kLCDM[kindex]/EDE.h(),zDES[zindex])
        pkzLCDM[kindex,zindex] = LCDM.pk(kLCDM[kindex]/LCDM.h(),zDES[zindex])
        pkzEDE[kindex,zindex] = EDE.pk(kLCDM[kindex]/EDE.h(),zDES[zindex])
        pkzDiff[kindex,zindex] = (pkzEDE[kindex,zindex]*pkzLCDM[kindex,zindex]) - 1.
        pkzratio[kindex,zindex] = pkzEDE[kindex,zindex]/pkzLCDM[kindex,zindex]

In [39]:
#z=0: P(k) in in units of (Mpc)^-3 and k in units of (h/Mpc)

pkzLCDM0 = np.zeros(len(kLCDM)-2)
pkzEDE0 = np.zeros(len(kLCDM)-2)
pkzDiff0 = np.zeros(len(kLCDM)-2)
pkzratio0 = np.zeros(len(kLCDM)-2)
for kindex in range(len(kLCDM)-2):
#        pkzLCDM0[kindex] = LCDM.h()**3*LCDM.pk(kLCDM[kindex]/LCDM.h(),0)
#        pkzEDE0[kindex] = EDE.h()**3*EDE.pk(kLCDM[kindex]/EDE.h(),0)
        pkzLCDM0[kindex] = LCDM.pk(kLCDM[kindex]/LCDM.h(),0)
        pkzEDE0[kindex] = EDE.pk(kLCDM[kindex]/EDE.h(),0)

        pkzDiff0[kindex] = (pkzEDE0[kindex]/pkzLCDM0[kindex]) - 1.
        pkzratio0[kindex] = pkzEDE0[kindex]/pkzLCDM0[kindex]

In [36]:
# P(k) comparison in units of h/Mpc
plt.figure()
plt.plot(kLCDMhLCDM,pkzLCDM0,c='k',lw=1,label='LCDM; h=.68')
plt.plot(kLCDMhEDE,pkzEDE0,c='r',lw=1,label='EDE; h=.72')
plt.ylabel(r'$P(k) [(h \, {\rm Mpc}^{-1})^{-3}]$ ')
plt.xlabel(r'$k [h\, {\rm Mpc}^{-1}]$')
plt.yscale('log')
plt.xscale('log')
plt.ylim([1e2,1e5])
plt.xlim([1e-4,1e1])
plt.title('$P(k)$ comparison in units of h/Mpc at z=0')
leg = plt.legend(frameon=True);
plt.savefig('Pkh.pdf')
plt.clf()

In [24]:
# 5-panel plot: Non-Linear Matter Power Spectrum at the DES redshifts



fig, axs = plt.subplots(1, 5, sharey=True)

fig.subplots_adjust(hspace=0)

#plt.subplot(1, 5, 1)
axs[0].plot(kLCDM,pkzLCDM[:,0],c='k',lw=1,label='LCDM')
axs[0].plot(kLCDM,pkzEDE[:,0],c='r',lw=1,label='EDE')
axs[0].set_ylabel(r'$P(k)$')
#axs[0].set_xlabel(r'k, Mpc$^{-1}$')
axs[0].set_yscale('log')
axs[0].set_xscale('log')
axs[0].set_ylim([1e2,1e5])
axs[0].set_xlim([1e-4,1e1])
axs[0].set_title('z=0.225')

axs[1].plot(kLCDM,pkzLCDM[:,1],c='k',lw=1)
axs[1].plot(kLCDM,pkzEDE[:,1],c='r',lw=1)
axs[1].set_yscale('log')
axs[1].set_xscale('log')
axs[1].set_ylim([1e2,1e5])
axs[1].set_xlim([1e-4,1e1])
axs[1].set_title('z=0.375')

axs[2].plot(kLCDM,pkzLCDM[:,2],c='k',lw=1)
axs[2].plot(kLCDM,pkzEDE[:,2],c='r',lw=1)
axs[2].set_yscale('log')
axs[2].set_xscale('log')
axs[2].set_ylim([1e2,1e5])
axs[2].set_xlim([1e-4,1e1])
axs[2].set_title('z=0.525')

axs[3].plot(kLCDM,pkzLCDM[:,3],c='k',lw=1)
axs[3].plot(kLCDM,pkzEDE[:,3],c='r',lw=1)
axs[3].set_yscale('log')
axs[3].set_xscale('log')
axs[3].set_ylim([1e2,1e5])
axs[3].set_xlim([1e-4,1e1])
axs[3].set_title('z=0.675')

axs[4].plot(kLCDM,pkzLCDM[:,4],c='k',lw=1,label='LCDM')
axs[4].plot(kLCDM,pkzEDE[:,4],c='r',lw=1,label='EDE')
axs[4].set_yscale('log')
axs[4].set_xscale('log')
axs[4].set_ylim([1e2,1e5])
axs[4].set_xlim([1e-4,1e1])
axs[4].set_title('z=0.825')
leg = axs[4].legend(frameon=False);



plt.savefig('Pk.pdf') 


ValueError: x and y must have same first dimension

In [104]:
# 5-panel Fractional Difference Plot

# rescaled k
kLCDMhLCDM=kLCDM/LCDM.h()
kLCDMhEDE=kLCDM/EDE.h()

figdiff, axsdiff = plt.subplots(1, 5, sharey=True)

figdiff.subplots_adjust(hspace=0)

#plt.subplot(1, 5, 1)
axsdiff[0].plot(kLCDMhLCDM,pkzDiff[:,0],c='k',lw=1)
axsdiff[0].set_ylabel(r'$1 \, -\, (P(k)_{EDE}/P(k)_{LCDM})$')
#axsdiff[0].set_xlabel(r'k, Mpc$^{-1}$')
axsdiff[0].set_xscale('log')
axsdiff[0].set_ylim([-.3,0.1])
axsdiff[0].set_xlim([1e-4,1e1])

axsdiff[1].plot(kLCDM,pkzDiff[:,1],c='k',lw=1)
axsdiff[1].set_xscale('log')
axsdiff[1].set_xlim([1e-4,1e1])

axsdiff[2].plot(kLCDM,pkzDiff[:,2],c='k',lw=1)
axsdiff[2].set_xscale('log')
axsdiff[2].set_xlim([1e-4,1e1])

axsdiff[3].plot(kLCDM,pkzDiff[:,3],c='k',lw=1)
axsdiff[3].set_xscale('log')
axsdiff[3].set_xlim([1e-4,1e1])

axsdiff[4].plot(kLCDM,pkzDiff[:,4],c='k',lw=1)
axsdiff[4].set_xscale('log')
axsdiff[4].set_xlim([1e-4,1e1])


plt.savefig('diffPk.pdf') 

In [11]:
# fEDE 

baEDE = EDE.get_background()
fEDE = EDE.fEDE()
z_c = EDE.z_c()
baH = baEDE['H [1/Mpc]']
baT = baEDE['conf. time [Mpc]']
baa = 1/(1 + baEDE['z'])
bV = baEDE['V_e_scf']
bpp = baEDE["phi'_scf"]
baCrit = baEDE['(.)rho_crit']
rho_scf = (bpp*bpp/(2*baa*baa) + bV)/3.
plt.figure(figsize=(10,10))
plt.rcParams.update({'font.size': 18})
plt.plot(baEDE['z'],rho_scf/baCrit,lw=2,c='k')
#plt.axhline(fEDE,c='r',ls='--')
#plt.axvline(z_c,c='r',ls='--')
plt.xscale('log')
plt.xlim([10,1e5])
plt.xlabel('z')
plt.ylabel(r'$f_{EDE}$')
plt.savefig('fEDE.pdf')
plt.clf()

In [43]:
print math.log10(EDE.z_c())
print EDE.fEDE()
print LCDM.h()
print EDE.h()

3.56273242375
0.122367935724
0.6821
0.7222366


In [41]:
# single-panel non-Linear P(k) fractional difference



plt.figure(figsize=(12,12))
plt.plot(kLCDMhLCDM,pkzDiff0,color="0.1",lw=1,label='z=0')
plt.plot(kLCDMhLCDM,pkzDiff[:,0],color="0.2",lw=1,label='z=0.225')
plt.plot(kLCDMhLCDM,pkzDiff[:,1],color="0.3",lw=1,label='z=0.375')
plt.plot(kLCDMhLCDM,pkzDiff[:,2],color="0.5",lw=1,label='z=0.525')
plt.plot(kLCDMhLCDM,pkzDiff[:,3],color="0.7",lw=1,label='z=0.675')
plt.plot(kLCDMhLCDM,pkzDiff[:,4],color="0.8",lw=1,label='z=0.825')
plt.hlines(0, 1e-4, 10, colors='k', linestyles='dashed')
plt.grid()
plt.xscale('log')
plt.xlabel(r'$k\,  [{\rm Mpc}^{-1}]$')
plt.ylabel(r'$ P(k)_{EDE}/P(k)_{LCDM} \, - \,1$')
plt.xlim([1e-4,1e1])
plt.ylim([-.25,0.05])
plt.vlines(1./(8./LCDM.h()), -.25, .05, colors='r', linestyles='dashed',label='h/8 ')
plt.legend(loc='upper center')
plt.savefig('PkDiffOnePloth.pdf')
plt.clf()


plt.figure(figsize=(12,12))
plt.plot(kLCDMhLCDM,pkzratio0,color="0.1",lw=1,label='z=0')
plt.plot(kLCDMhLCDM,pkzratio[:,0],color="0.2",lw=1,label='z=0.225')
plt.plot(kLCDMhLCDM,pkzratio[:,1],color="0.3",lw=1,label='z=0.375')
plt.plot(kLCDMhLCDM,pkzratio[:,2],color="0.5",lw=1,label='z=0.525')
plt.plot(kLCDMhLCDM,pkzratio[:,3],color="0.7",lw=1,label='z=0.675')
plt.plot(kLCDMhLCDM,pkzratio[:,4],color="0.8",lw=1,label='z=0.825')
plt.hlines(1, 1e-4, 10, colors='k', linestyles='dashed')
plt.vlines(1./(8./LCDM.h()), .5, 1.5, colors='r', linestyles='dashed',label='h/8 ')
plt.grid()
plt.xscale('log')
plt.xlabel(r'$k\,  [h {\rm Mpc}^{-1}]$')
plt.ylabel(r'$ P(k)_{EDE} [{\rm Mpc}^{3}] / P(k)_{LCDM}[{\rm Mpc}^{3}] $')
plt.xlim([1e-4,1e1])
plt.ylim([.5,1.5])
plt.legend(loc='upper center')
plt.savefig('PkDiffOnePlot-ratioh.pdf')
plt.clf()


In [13]:
# single-panel non-Linear P(k)

plt.figure(figsize=(12,12))
plt.plot(kLCDM,pkzEDE[:,0],c='r',lw=1)
plt.plot(kLCDM,pkzEDE[:,1],c='r',lw=1)
plt.plot(kLCDM,pkzEDE[:,2],c='r',lw=1)
plt.plot(kLCDM,pkzEDE[:,3],c='r',lw=1)
plt.plot(kLCDM,pkzEDE[:,4],c='r',lw=1)
plt.plot(kLCDM,pkzLCDM[:,0],c='k',lw=1)
plt.plot(kLCDM,pkzLCDM[:,1],c='k',lw=1)
plt.plot(kLCDM,pkzLCDM[:,2],c='k',lw=1)
plt.plot(kLCDM,pkzLCDM[:,3],c='k',lw=1)
plt.plot(kLCDM,pkzLCDM[:,4],c='k',lw=1)
plt.xscale('log')
plt.yscale('log')
plt.savefig('PkOnePlot.pdf')

# Consider varying fEDE

In [14]:
EDE1 = Class()
EDE1.set({'f_scf': 3.98e+26, 'm_scf': 5.31e-28,'thetai_scf': 2.83, 'A_s': 2.215e-09, 'n_s': 0.9889, '100*theta_s': 1.04152, 'omega_b': 0.02253, 'omega_cdm': 0.1306, 'm_ncdm': 0.06, 'tau_reio': 0.072})
EDE1.set({'non linear':'HMCODE','N_ncdm':1, 'N_ur':2.0328, 'Omega_Lambda':0.0, 'Omega_fld':0, 'Omega_scf':-1, 'n_scf':3, 'CC_scf':1, 'scf_parameters':'1, 1, 1, 1, 1, 0.0', 'scf_tuning_index':3, 'attractor_ic_scf':'no', 'output':'tCl pCl lCl mPk', 'lensing':'yes', 'l_max_scalars':2508, 'P_k_max_h/Mpc':200,'z_max_pk':2.})
EDE1.compute()
EDE2 = Class()
EDE2.set({'f_scf': 4.98e+26, 'm_scf': 5.31e-28,'thetai_scf': 2.83, 'A_s': 2.215e-09, 'n_s': 0.9889, '100*theta_s': 1.04152, 'omega_b': 0.02253, 'omega_cdm': 0.1306, 'm_ncdm': 0.06, 'tau_reio': 0.072})
EDE2.set({'non linear':'HMCODE','N_ncdm':1, 'N_ur':2.0328, 'Omega_Lambda':0.0, 'Omega_fld':0, 'Omega_scf':-1, 'n_scf':3, 'CC_scf':1, 'scf_parameters':'1, 1, 1, 1, 1, 0.0', 'scf_tuning_index':3, 'attractor_ic_scf':'no', 'output':'tCl pCl lCl mPk', 'lensing':'yes', 'l_max_scalars':2508, 'P_k_max_h/Mpc':200,'z_max_pk':2.})
EDE2.compute()
EDE3 = Class()
EDE3.set({'f_scf': 5.98e+26, 'm_scf': 5.31e-28,'thetai_scf': 2.83, 'A_s': 2.215e-09, 'n_s': 0.9889, '100*theta_s': 1.04152, 'omega_b': 0.02253, 'omega_cdm': 0.1306, 'm_ncdm': 0.06, 'tau_reio': 0.072})
EDE3.set({'non linear':'HMCODE','N_ncdm':1, 'N_ur':2.0328, 'Omega_Lambda':0.0, 'Omega_fld':0, 'Omega_scf':-1, 'n_scf':3, 'CC_scf':1, 'scf_parameters':'1, 1, 1, 1, 1, 0.0', 'scf_tuning_index':3, 'attractor_ic_scf':'no', 'output':'tCl pCl lCl mPk', 'lensing':'yes', 'l_max_scalars':2508, 'P_k_max_h/Mpc':200,'z_max_pk':2.})
EDE3.compute()
EDE4 = Class()
EDE4.set({'f_scf': 6.98e+26, 'm_scf': 5.31e-28,'thetai_scf': 2.83, 'A_s': 2.215e-09, 'n_s': 0.9889, '100*theta_s': 1.04152, 'omega_b': 0.02253, 'omega_cdm': 0.1306, 'm_ncdm': 0.06, 'tau_reio': 0.072})
EDE4.set({'non linear':'HMCODE','N_ncdm':1, 'N_ur':2.0328, 'Omega_Lambda':0.0, 'Omega_fld':0, 'Omega_scf':-1, 'n_scf':3, 'CC_scf':1, 'scf_parameters':'1, 1, 1, 1, 1, 0.0', 'scf_tuning_index':3, 'attractor_ic_scf':'no', 'output':'tCl pCl lCl mPk', 'lensing':'yes', 'l_max_scalars':2508, 'P_k_max_h/Mpc':200,'z_max_pk':2.})
EDE4.compute()


In [35]:
print EDE1.fEDE()
print EDE2.fEDE()
print EDE3.fEDE()
print EDE4.fEDE()

0.122367935724
0.183896831053
0.252522936444
0.32533009618


In [15]:
 
pkzEDE1 = np.zeros((len(kLCDM),len(zDES)))
pkzEDE2 = np.zeros((len(kLCDM),len(zDES)))
pkzEDE3 = np.zeros((len(kLCDM),len(zDES)))
pkzEDE4 = np.zeros((len(kLCDM),len(zDES)))
pkzDiff1 = np.zeros((len(kLCDM),len(zDES)))
pkzDiff2 = np.zeros((len(kLCDM),len(zDES)))
pkzDiff3 = np.zeros((len(kLCDM),len(zDES)))
pkzDiff4 = np.zeros((len(kLCDM),len(zDES)))
for kindex in range(len(kLCDM)):
    for zindex in range(len(zDES)):
        pkzEDE1[kindex,zindex] = EDE1.pk(kLCDM[kindex],zDES[zindex])
        pkzDiff1[kindex,zindex] = (-pkzLCDM[kindex,zindex] + pkzEDE1[kindex,zindex])/pkzLCDM[kindex,zindex]
        pkzEDE2[kindex,zindex] = EDE2.pk(kLCDM[kindex],zDES[zindex])
        pkzDiff2[kindex,zindex] = (-pkzLCDM[kindex,zindex] + pkzEDE2[kindex,zindex])/pkzLCDM[kindex,zindex]
        pkzEDE3[kindex,zindex] = EDE3.pk(kLCDM[kindex],zDES[zindex])
        pkzDiff3[kindex,zindex] = (-pkzLCDM[kindex,zindex] + pkzEDE3[kindex,zindex])/pkzLCDM[kindex,zindex]
        pkzEDE4[kindex,zindex] = EDE4.pk(kLCDM[kindex],zDES[zindex])
        pkzDiff4[kindex,zindex] = (-pkzLCDM[kindex,zindex] + pkzEDE4[kindex,zindex])/pkzLCDM[kindex,zindex]

In [19]:
# Plot diffPk for varying fEDE
plt.clf()
plt.figure(figsize=(12,12))
plt.plot(kLCDM,pkzDiff1[:,0],color="0.2",lw=1,label='$f_{EDE}=.12$')
plt.plot(kLCDM,pkzDiff2[:,0],color="0.4",lw=1,label='$f_{EDE}=.18$')
plt.plot(kLCDM,pkzDiff3[:,0],color="0.7",lw=1,label='$f_{EDE}=.25$')
plt.plot(kLCDM,pkzDiff4[:,0],color="0.9",lw=1,label='$f_{EDE}=.32$')
plt.xscale('log')
plt.xlim([1e-4,1e1])
plt.ylim([-.5,0.05])
plt.ylabel(r'$(P(k)_{LCDM}-P(k)_{EDE})/P(k)_{LCDM}$')
plt.legend(loc='upper center')
plt.savefig('PkDiff_fEDE.pdf')



In [156]:
# For reference: 5-panel Linear-Power Spectrum

pkzlinLCDM = np.zeros((len(kLCDM),len(zDES)))
pkzlinEDE = np.zeros((len(kLCDM),len(zDES)))
pkzlinDiff= np.zeros((len(kLCDM),len(zDES)))
for kindex in range(len(kLCDM)):
    for zindex in range(len(zDES)):
        pkzlinLCDM[kindex,zindex] = LCDM.pk_lin(kLCDM[kindex],zDES[zindex])
        pkzlinEDE[kindex,zindex] = EDE.pk_lin(kLCDM[kindex],zDES[zindex])
        pkzlinDiff[kindex,zindex] = (pkzlinLCDM[kindex,zindex] - pkzlinEDE[kindex,zindex])/pkzlinLCDM[kindex,zindex]

figlin, axslin = plt.subplots(1, 5, sharey=True)

figlin.subplots_adjust(hspace=0)

#plt.subplot(1, 5, 1)
axslin[0].plot(kLCDM,pkzlinLCDM[:,0],c='k',lw=1)
axslin[0].plot(kLCDM,pkzlinEDE[:,0],c='r',lw=1)
axslin[0].set_ylabel(r'$P(k)$')
#axslin[0].set_xlabel(r'k, Mpc$^{-1}$')
axslin[0].set_yscale('log')
axslin[0].set_xscale('log')
axslin[0].set_ylim([1e2,1e5])
axslin[0].set_xlim([1e-4,1e0])

axslin[1].plot(kLCDM,pkzlinLCDM[:,1],c='k',lw=1)
axslin[1].plot(kLCDM,pkzlinEDE[:,1],c='r',lw=1)
axslin[1].set_yscale('log')
axslin[1].set_xscale('log')
axslin[1].set_ylim([1e2,1e5])
axslin[1].set_xlim([1e-4,1e0])

axslin[2].plot(kLCDM,pkzlinLCDM[:,2],c='k',lw=1)
axslin[2].plot(kLCDM,pkzlinEDE[:,2],c='r',lw=1)
axslin[2].set_yscale('log')
axslin[2].set_xscale('log')
axslin[2].set_ylim([1e2,1e5])
axslin[2].set_xlim([1e-4,1e0])

axslin[3].plot(kLCDM,pkzlinLCDM[:,3],c='k',lw=1)
axslin[3].plot(kLCDM,pkzlinEDE[:,3],c='r',lw=1)
axslin[3].set_yscale('log')
axslin[3].set_xscale('log')
axslin[3].set_ylim([1e2,1e5])
axslin[3].set_xlim([1e-4,1e0])

axslin[4].plot(kLCDM,pkzlinLCDM[:,4],c='k',lw=1)
axslin[4].plot(kLCDM,pkzlinEDE[:,4],c='r',lw=1)
axslin[4].set_yscale('log')
axslin[4].set_xscale('log')
axslin[4].set_ylim([1e2,1e5])
axslin[4].set_xlim([1e-4,1e0])


plt.savefig('Pklin.pdf')

figlindiff, axslindiff = plt.subplots(1, 5, sharey=True)

figlindiff.subplots_adjust(hspace=0)

#plt.subplot(1, 5, 1)
axslindiff[0].plot(kLCDM,pkzlinDiff[:,0],c='k',lw=1)
axslindiff[0].set_ylabel(r'$P(k)$')
#axslindiff[0].set_xlabel(r'k, Mpc$^{-1}$')
axslindiff[0].set_xscale('log')
axslindiff[0].set_ylim([-1,1])
axslindiff[0].set_xlim([1e-4,1e0])

axslindiff[1].plot(kLCDM,pkzlinDiff[:,1],c='k',lw=1)
axslindiff[1].set_xscale('log')
axslindiff[0].set_ylim([-1,1])
axslindiff[1].set_xlim([1e-4,1e0])

axslindiff[2].plot(kLCDM,pkzlinDiff[:,2],c='k',lw=1)
axslindiff[2].set_xscale('log')
axslindiff[0].set_ylim([-1,1])
axslindiff[2].set_xlim([1e-4,1e0])

axslindiff[3].plot(kLCDM,pkzlinDiff[:,3],c='k',lw=1)
axslindiff[3].set_xscale('log')
axslindiff[0].set_ylim([-1,0.1])
axslindiff[3].set_xlim([1e-4,1e0])

axslindiff[4].plot(kLCDM,pkzlinDiff[:,4],c='k',lw=1)
axslindiff[4].set_xscale('log')
axslindiff[0].set_ylim([-1,1])
axslindiff[4].set_xlim([1e-4,1e0])


plt.savefig('diffPklin.pdf')


In [80]:
EDE.Om_m(0)

0.2950722535684526